In [0]:
dbutils.secrets.list('scope-to-key-vault')

[SecretMetadata(key='app-id'),
 SecretMetadata(key='app-value'),
 SecretMetadata(key='dbpassword'),
 SecretMetadata(key='DBurl'),
 SecretMetadata(key='dbusername'),
 SecretMetadata(key='event-hub-name'),
 SecretMetadata(key='event-hub-sas'),
 SecretMetadata(key='weather-api-key')]

In [0]:
from azure.eventhub import *
import json

In [0]:
eventhub_name = 'weatherstreamingeventhub'

In [0]:
producer = EventHubProducerClient.from_connection_string(dbutils.secrets.get("scope-to-key-vault", "event-hub-sas"), eventhub_name=eventhub_name)



In [0]:
# function to send events to event hub

def send_event(event):
    data_event_batch =producer.create_batch()
    data_event_batch.add(EventData(json.dumps(event)))
    producer.send_batch(data_event_batch)


# sample JSON data
event= {
    "event_id" : 1111,
    "event_name" : "test_event",
}

#send event
send_event(event)

#close the producer
producer.close()

In [0]:
producer = EventHubProducerClient.from_connection_string(conn_str='Endpoint=sb://weatherstreamingnamespace-dev.servicebus.windows.net/;SharedAccessKeyName=databricks-connection-access-sas;SharedAccessKey=kNd2lKWccWa6NbUvG/0puYQ/3/1vTM0oA+AEhOpckUA=;EntityPath=weatherstreamingeventhub', eventhub_name='weatherstreamingeventhub')



#API Testing

In [0]:
import requests
import json

weatherapikey = dbutils.secrets.get("scope-to-key-vault", "weather-api-key")
location="Chennai"

base_url ="http://api.weatherapi.com/v1/"
current_weather_url = f"{base_url}/current.json"

params={
    'key': weatherapikey,
    'q' : location,
}

response = requests.get(current_weather_url, params=params)

if response.status_code == 200:
    current_weather = response.json()
    print("Current Weather:")
    print(json.dumps(current_weather, indent=3))
else:
    print(f"Error: {response.status_code}, {response.text}")







Current Weather:
{
   "location": {
      "name": "Chennai",
      "region": "Tamil Nadu",
      "country": "India",
      "lat": 13.0833,
      "lon": 80.2833,
      "tz_id": "Asia/Kolkata",
      "localtime_epoch": 1742970613,
      "localtime": "2025-03-26 12:00"
   },
   "current": {
      "last_updated_epoch": 1742969700,
      "last_updated": "2025-03-26 11:45",
      "temp_c": 33.3,
      "temp_f": 91.9,
      "is_day": 1,
      "condition": {
         "text": "Partly cloudy",
         "icon": "//cdn.weatherapi.com/weather/64x64/day/116.png",
         "code": 1003
      },
      "wind_mph": 10.7,
      "wind_kph": 17.3,
      "wind_degree": 131,
      "wind_dir": "SE",
      "pressure_mb": 1009.0,
      "pressure_in": 29.8,
      "precip_mm": 0.0,
      "precip_in": 0.0,
      "humidity": 59,
      "cloud": 50,
      "feelslike_c": 38.4,
      "feelslike_f": 101.1,
      "windchill_c": 30.8,
      "windchill_f": 87.4,
      "heatindex_c": 33.3,
      "heatindex_f": 91.9,
      "

#Complete Code for getting all The Weather data

In [0]:
import requests
import json

# Function to handle the API response
def handle_response(response):
    if response.status_code == 200:
        return response.json()
    else:
        return f"Error: {response.status_code}, {response.text}"

# Function to get current weather and air quality data
def get_current_weather(base_url, api_key, location):
    current_weather_url = f"{base_url}/current.json"
    params = {
        'key': api_key,
        'q': location,
        "aqi": 'yes'
    }
    response = requests.get(current_weather_url, params=params)
    return handle_response(response)

# Function to get Forecast Data
def get_forecast_weather(base_url, api_key, location, days):
    forecast_url = f"{base_url}/forecast.json"
    params = {
        "key": api_key,
        "q": location,
        "days": days,
    }
    response = requests.get(forecast_url, params=params)
    return handle_response(response)

# Function to get Alerts
def get_alerts(base_url, api_key, location):
    alerts_url = f"{base_url}/alerts.json"
    params = {
        'key': api_key,
        'q': location,
        "alerts": 'yes'
    }
    response = requests.get(alerts_url, params=params)
    return handle_response(response)

# Flatten and merge the data
def flatten_data(current_weather, forecast_weather, alerts):
    location_data = current_weather.get("location", {})
    current = current_weather.get("current", {})
    condition = current.get("condition", {})
    air_quality = current.get("air_quality", {})
    forecast = forecast_weather.get("forecast", {}).get("forecastday", [])
    alert_list = alerts.get("alerts", {}).get("alert", [])

    flattened_data = {
        'name': location_data.get('name'),
        'region': location_data.get('region'),
        'country': location_data.get('country'),
        'lat': location_data.get('lat'),
        'lon': location_data.get('lon'),
        'localtime': location_data.get('localtime'),
        'temp_c': current.get('temp_c'),
        'is_day': current.get('is_day'),
        'condition_text': condition.get('text'),
        'condition_icon': condition.get('icon'),
        'wind_kph': current.get('wind_kph'),
        'wind_degree': current.get('wind_degree'),
        'wind_dir': current.get('wind_dir'),
        'pressure_in': current.get('pressure_in'),
        'precip_in': current.get('precip_in'),
        'humidity': current.get('humidity'),
        'cloud': current.get('cloud'),
        'feelslike_c': current.get('feelslike_c'),
        'uv': current.get('uv'),
        'air_quality': {
            'co': air_quality.get('co'),
            'no2': air_quality.get('no2'),
            'o3': air_quality.get('o3'),
            'so2': air_quality.get('so2'),
            'pm2_5': air_quality.get('pm2_5'),
            'pm10': air_quality.get('pm10'),
            'us-epa-index': air_quality.get('us-epa-index'),
            'gb-defra-index': air_quality.get('gb-defra-index')
        },
        'alerts': [
            {
                'headline': alert.get('headline'),
                'severity': alert.get('severity'),
                'description': alert.get('desc'),
                'instruction': alert.get('instruction')
            }
            for alert in alert_list
        ],
        'forecast': [
            {
                'date': day.get('date'),
                'maxtemp_c': day.get('day', {}).get('maxtemp_c'),
                'mintemp_c': day.get('day', {}).get('mintemp_c'),
                'condition': day.get('day', {}).get('condition', {}).get('text')
            }
            for day in forecast
        ]
    }
    return flattened_data

# Main program
def fetch_weather_data():

    base_url = "http://api.weatherapi.com/v1/"
    location = "Chennai"  # You can replace with any city name based on your preference
    weatherapikey = dbutils.secrets.get("scope-to-key-vault", "weather-api-key")
    

    # Get data from API
    current_weather = get_current_weather(base_url, weatherapikey, location)
    forecast_weather = get_forecast_weather(base_url, weatherapikey, location, 3)
    alerts = get_alerts(base_url, weatherapikey, location)

    # Flatten and merge data
    merged_data = flatten_data(current_weather, forecast_weather, alerts)
    print("Weather Data:", json.dumps(merged_data, indent=3))

# Calling the Main Program
fetch_weather_data()

Weather Data: {
   "name": "Chennai",
   "region": "Tamil Nadu",
   "country": "India",
   "lat": 13.0833,
   "lon": 80.2833,
   "localtime": "2025-03-26 12:10",
   "temp_c": 33.3,
   "is_day": 1,
   "condition_text": "Partly cloudy",
   "condition_icon": "//cdn.weatherapi.com/weather/64x64/day/116.png",
   "wind_kph": 20.9,
   "wind_degree": 126,
   "wind_dir": "SE",
   "pressure_in": 29.8,
   "precip_in": 0.0,
   "humidity": 59,
   "cloud": 50,
   "feelslike_c": 39.3,
   "uv": 11.5,
   "air_quality": {
      "co": 801.05,
      "no2": 20.72,
      "o3": 77.0,
      "so2": 11.655,
      "pm2_5": 59.94,
      "pm10": 69.19,
      "us-epa-index": 3,
      "gb-defra-index": 8
   },
   "alerts": [],
   "forecast": [
      {
         "date": "2025-03-26",
         "maxtemp_c": 30.8,
         "mintemp_c": 25.7,
         "condition": "Sunny"
      },
      {
         "date": "2025-03-27",
         "maxtemp_c": 33.0,
         "mintemp_c": 25.4,
         "condition": "Sunny"
      },
      {
 

#Sending complete weather DATA to Event HUB

In [0]:
import requests
import json
from azure.eventhub import EventHubProducerClient, EventData

# Event Hub configuration

# Getting secret value from Key Vault
eventhub_connection_string = dbutils.secrets.get("scope-to-key-vault", "event-hub-sas")

EVENT_HUB_NAME = "weatherstreamingeventhub"

# Initialize the Event Hub producer
producer = EventHubProducerClient.from_connection_string(conn_str=eventhub_connection_string, eventhub_name=EVENT_HUB_NAME)

# Function to send events to Event Hub
def send_event(event):
    event_data_batch = producer.create_batch()
    event_data_batch.add(EventData(json.dumps(event)))
    producer.send_batch(event_data_batch)

# Function to handle the API response
def handle_response(response):
    if response.status_code == 200:
        return response.json()
    else:
        return f"Error: {response.status_code}, {response.text}"

# Function to get current weather and air quality data
def get_current_weather(base_url, api_key, location):
    current_weather_url = f"{base_url}/current.json"
    params = {
        'key': api_key,
        'q': location,
        "aqi": 'yes'
    }
    response = requests.get(current_weather_url, params=params)
    return handle_response(response)

# Function to get Forecast Data
def get_forecast_weather(base_url, api_key, location, days):
    forecast_url = f"{base_url}/forecast.json"
    params = {
        "key": api_key,
        "q": location,
        "days": days,
    }
    response = requests.get(forecast_url, params=params)
    return handle_response(response)

# Function to get Alerts
def get_alerts(base_url, api_key, location):
    alerts_url = f"{base_url}/alerts.json"
    params = {
        'key': api_key,
        'q': location,
        "alerts": 'yes'
    }
    response = requests.get(alerts_url, params=params)
    return handle_response(response)

# Flatten and merge the data
def flatten_data(current_weather, forecast_weather, alerts):
    location_data = current_weather.get("location", {})
    current = current_weather.get("current", {})
    condition = current.get("condition", {})
    air_quality = current.get("air_quality", {})
    forecast = forecast_weather.get("forecast", {}).get("forecastday", [])
    alert_list = alerts.get("alerts", {}).get("alert", [])

    flattened_data = {
        'name': location_data.get('name'),
        'region': location_data.get('region'),
        'country': location_data.get('country'),
        'lat': location_data.get('lat'),
        'lon': location_data.get('lon'),
        'localtime': location_data.get('localtime'),
        'temp_c': current.get('temp_c'),
        'is_day': current.get('is_day'),
        'condition_text': condition.get('text'),
        'condition_icon': condition.get('icon'),
        'wind_kph': current.get('wind_kph'),
        'wind_degree': current.get('wind_degree'),
        'wind_dir': current.get('wind_dir'),
        'pressure_in': current.get('pressure_in'),
        'precip_in': current.get('precip_in'),
        'humidity': current.get('humidity'),
        'cloud': current.get('cloud'),
        'feelslike_c': current.get('feelslike_c'),
        'uv': current.get('uv'),
        'air_quality': {
            'co': air_quality.get('co'),
            'no2': air_quality.get('no2'),
            'o3': air_quality.get('o3'),
            'so2': air_quality.get('so2'),
            'pm2_5': air_quality.get('pm2_5'),
            'pm10': air_quality.get('pm10'),
            'us-epa-index': air_quality.get('us-epa-index'),
            'gb-defra-index': air_quality.get('gb-defra-index')
        },
        'alerts': [
            {
                'headline': alert.get('headline'),
                'severity': alert.get('severity'),
                'description': alert.get('desc'),
                'instruction': alert.get('instruction')
            }
            for alert in alert_list
        ],
        'forecast': [
            {
                'date': day.get('date'),
                'maxtemp_c': day.get('day', {}).get('maxtemp_c'),
                'mintemp_c': day.get('day', {}).get('mintemp_c'),
                'condition': day.get('day', {}).get('condition', {}).get('text')
            }
            for day in forecast
        ]
    }
    return flattened_data

# Main program
def fetch_weather_data():

    base_url = "http://api.weatherapi.com/v1/"
    location = "Chennai"  # You can replace with any city name based on your preference
    weatherapikey = dbutils.secrets.get("scope-to-key-vault", "weather-api-key")

    # Get data from API
    current_weather = get_current_weather(base_url, weatherapikey, location)
    forecast_weather = get_forecast_weather(base_url, weatherapikey, location, 3)
    alerts = get_alerts(base_url, weatherapikey, location)

    # Flatten and merge data
    merged_data = flatten_data(current_weather, forecast_weather, alerts)

    # Sending the weather data to Event HUB
    send_event(merged_data)
    

# Calling the Main Program
fetch_weather_data()

#sending data in event hub in streaming fashion in 30 seconds

In [0]:
import requests
import json
from azure.eventhub import EventHubProducerClient, EventData
from datetime import datetime, timedelta
# Event Hub configuration

# Getting secret value from Key Vault
eventhub_connection_string = dbutils.secrets.get("scope-to-key-vault", "event-hub-sas")

EVENT_HUB_NAME = "weatherstreamingeventhub"

# Initialize the Event Hub producer
producer = EventHubProducerClient.from_connection_string(conn_str=eventhub_connection_string, eventhub_name=EVENT_HUB_NAME)

# Function to send events to Event Hub
def send_event(event):
    event_data_batch = producer.create_batch()
    event_data_batch.add(EventData(json.dumps(event)))
    producer.send_batch(event_data_batch)

# Function to handle the API response
def handle_response(response):
    if response.status_code == 200:
        return response.json()
    else:
        return f"Error: {response.status_code}, {response.text}"

# Function to get current weather and air quality data
def get_current_weather(base_url, api_key, location):
    current_weather_url = f"{base_url}/current.json"
    params = {
        'key': api_key,
        'q': location,
        "aqi": 'yes'
    }
    response = requests.get(current_weather_url, params=params)
    return handle_response(response)

# Function to get Forecast Data
def get_forecast_weather(base_url, api_key, location, days):
    forecast_url = f"{base_url}/forecast.json"
    params = {
        "key": api_key,
        "q": location,
        "days": days,
    }
    response = requests.get(forecast_url, params=params)
    return handle_response(response)

# Function to get Alerts
def get_alerts(base_url, api_key, location):
    alerts_url = f"{base_url}/alerts.json"
    params = {
        'key': api_key,
        'q': location,
        "alerts": 'yes'
    }
    response = requests.get(alerts_url, params=params)
    return handle_response(response)

# Flatten and merge the data
def flatten_data(current_weather, forecast_weather, alerts):
    location_data = current_weather.get("location", {})
    current = current_weather.get("current", {})
    condition = current.get("condition", {})
    air_quality = current.get("air_quality", {})
    forecast = forecast_weather.get("forecast", {}).get("forecastday", [])
    alert_list = alerts.get("alerts", {}).get("alert", [])

    flattened_data = {
        'name': location_data.get('name'),
        'region': location_data.get('region'),
        'country': location_data.get('country'),
        'lat': location_data.get('lat'),
        'lon': location_data.get('lon'),
        'localtime': location_data.get('localtime'),
        'temp_c': current.get('temp_c'),
        'is_day': current.get('is_day'),
        'condition_text': condition.get('text'),
        'condition_icon': condition.get('icon'),
        'wind_kph': current.get('wind_kph'),
        'wind_degree': current.get('wind_degree'),
        'wind_dir': current.get('wind_dir'),
        'pressure_in': current.get('pressure_in'),
        'precip_in': current.get('precip_in'),
        'humidity': current.get('humidity'),
        'cloud': current.get('cloud'),
        'feelslike_c': current.get('feelslike_c'),
        'uv': current.get('uv'),
        'air_quality': {
            'co': air_quality.get('co'),
            'no2': air_quality.get('no2'),
            'o3': air_quality.get('o3'),
            'so2': air_quality.get('so2'),
            'pm2_5': air_quality.get('pm2_5'),
            'pm10': air_quality.get('pm10'),
            'us-epa-index': air_quality.get('us-epa-index'),
            'gb-defra-index': air_quality.get('gb-defra-index')
        },
        'alerts': [
            {
                'headline': alert.get('headline'),
                'severity': alert.get('severity'),
                'description': alert.get('desc'),
                'instruction': alert.get('instruction')
            }
            for alert in alert_list
        ],
        'forecast': [
            {
                'date': day.get('date'),
                'maxtemp_c': day.get('day', {}).get('maxtemp_c'),
                'mintemp_c': day.get('day', {}).get('mintemp_c'),
                'condition': day.get('day', {}).get('condition', {}).get('text')
            }
            for day in forecast
        ]
    }
    return flattened_data


def fetch_weather_data():

    base_url = "http://api.weatherapi.com/v1/"
    location = "Chennai"  # You can replace with any city name based on your preference
    weatherapikey = dbutils.secrets.get("scope-to-key-vault", "weather-api-key")

    # Get data from API
    current_weather = get_current_weather(base_url, weatherapikey, location)
    forecast_weather = get_forecast_weather(base_url, weatherapikey, location, 3)
    alerts = get_alerts(base_url, weatherapikey, location)

    # Flatten and merge data
    merged_data = flatten_data(current_weather, forecast_weather, alerts)

    # returning the weather data main function
    return merged_data


# Function to process each batch of streaming data
last_sent_time = datetime.now() - timedelta(seconds=30)  # Initialize last sent time


# Main program
def process_batch(batch_df, batch_id):
    global last_sent_time
    try:
        # Get current time
        current_time = datetime.now()
        
        # Check if 30 seconds have passed since last event was sent
        if (current_time - last_sent_time).total_seconds() >= 30:
            # Fetch weather data
            weather_data = fetch_weather_data()
            
            # Send the weather data (current weather part)
            send_event(weather_data)

            # Update last sent time
            last_sent_time = current_time
            print(f'Event Sent at {last_sent_time}')

    except Exception as e:
        print(f"Error sending events in batch {batch_id}: {str(e)}")
        raise e

# Set up a streaming source (for example, rate source for testing purposes)
streaming_df = spark.readStream.format("rate").option("rowsPerSecond", 1).load()

# Write the streaming data using foreachBatch to send weather data to Event Hub
query = streaming_df.writeStream.foreachBatch(process_batch).start()

query.awaitTermination()

# Close the producer after termination
producer.close()
    


Event Sent at 2025-03-27 15:06:30.937189
Event Sent at 2025-03-27 15:07:01.023466
Event Sent at 2025-03-27 15:07:31.760185
Event Sent at 2025-03-27 15:08:01.897037
Event Sent at 2025-03-27 15:08:32.515858
Event Sent at 2025-03-27 15:09:02.918071
Event Sent at 2025-03-27 15:09:33.742331
Event Sent at 2025-03-27 15:10:04.993464
Event Sent at 2025-03-27 15:10:35.913828
Event Sent at 2025-03-27 15:11:06.782518
Event Sent at 2025-03-27 15:11:37.759313
Event Sent at 2025-03-27 15:12:08.494416
Event Sent at 2025-03-27 15:12:39.689297
Event Sent at 2025-03-27 15:13:10.843568
Event Sent at 2025-03-27 15:13:41.820082
Event Sent at 2025-03-27 15:14:12.556027
Event Sent at 2025-03-27 15:14:42.902330
Event Sent at 2025-03-27 15:15:13.672533
Event Sent at 2025-03-27 15:15:43.801082
Event Sent at 2025-03-27 15:16:13.905448
Event Sent at 2025-03-27 15:16:44.561309
Event Sent at 2025-03-27 15:17:15.776905
Event Sent at 2025-03-27 15:17:46.836275
Event Sent at 2025-03-27 15:18:17.634909
Event Sent at 20

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:136)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:136)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:133)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:133)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:715)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:435)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:435)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can